In [1]:
from time import time
import multiprocessing
import netCDF4 as nc
import xarray as xr
import numpy as np

In [2]:
range_combined = xr.open_mfdataset('data/raw/range-*.nc',combine='by_coords')

In [3]:
rho = 1025
g = 9.81
T = (12*60+25)*60
omega = np.pi*2/T 
phi = np.pi/4
A = 1
Cp = 1

epoch = np.datetime64('1970-01-01T00:00:00')

In [4]:
def peak_trough_idx(height,t,latitude,longitude):
    '''
    indices of the peaks and troughs
    
    '''
    secs = (t-epoch)/1e9 # dtype is timedelta[s] not timedelta[ns]
    
    step = 35
    idx_troughs = np.zeros(((int((secs[-1]-secs[0])/T)+1),len(latitude),len(longitude)))
    idx_peaks = np.zeros(((int((secs[-1]-secs[0])/T)+1),len(latitude),len(longitude)))
    idx_troughs[:] = np.nan
    idx_peaks[:] = np.nan
    
    
    
    
    for j, lat in enumerate(latitude):
        for k, long in enumerate(longitude):
            
            if np.isnan(height[0,j,k]) == True:
                pass
            
            if height[0,j,k] > height[1,j,k]:
                
                trough = min(height[:step,j,k])
                idx_trough = [x for x, z in enumerate(height[:step,j,k]) if z == trough][0]
                idx_troughs[0,j,k] = idx_trough

                peak = max(height[:step+step,j,k])
                idx_peak = [x for x, z in enumerate(height[:step+step,j,k]) if z == peak][0]
                idx_peaks[0,j,k] = idx_peak
                
                
                for i in range(1,int((secs[-1]-secs[0])/T)+1):

                    idx_trough += 25
                    trough_slice = height[idx_trough:idx_trough+step,j,k]
                    trough = min(trough_slice)
                    idx_trough = [x for x, z in enumerate(trough_slice) if z == trough][0] + idx_trough
                    idx_troughs[i,j,k] = idx_trough

                    idx_peak += 25
                    peak_slice = height[idx_peak:idx_peak+step,j,k]
                    peak = max(peak_slice)
                    idx_peak = [x for x, z in enumerate(peak_slice) if z == peak][0] + idx_peak
                    idx_peaks[i,j,k] = idx_peak
                
                
            elif height[0,j,k] < height[1,j,k]:
                
                peak = max(height[:step,j,k])
                idx_peak = [x for x, z in enumerate(height[:step,j,k]) if z == peak][0]
                idx_peaks[0,j,k] = idx_peak

                trough = min(height[:step+step,j,k])
                idx_trough = [x for x, z in enumerate(height[:step+step,j,k]) if z == trough][0]
                idx_troughs[0,j,k] = idx_trough


                for i in range(1,int((secs[-1]-secs[0])/T)+1):

                    idx_trough += 25
                    trough_slice = height[idx_trough:idx_trough+step,j,k]
                    trough = min(trough_slice)
                    idx_trough = [x for x, z in enumerate(trough_slice) if z == trough][0] + idx_trough
                    idx_troughs[i,j,k] = idx_trough

                    idx_peak += 25
                    peak_slice = height[idx_peak:idx_peak+step,j,k]
                    peak = max(peak_slice)
                    idx_peak = [x for x, z in enumerate(peak_slice) if z == peak][0] + idx_peak
                    idx_peaks[i,j,k] = idx_peak
                    
                    
                    
    return idx_peaks,idx_troughs

    
    
def write_peak_trough(idx_peaks,idx_troughs,latitude,longitude,filename):
    '''
    
    '''
    
#     idx_peaks = xr.DataArray(idx_peaks, 
#                              coords={'occurrences': np.arange(1,idx_peaks.shape[0]+1),
#                                      'lat': latitude,
#                                      'lon': longitude}, 
#                              dims=['occurrences','lat','lon'])
    
#     idx_troughs = xr.DataArray(idx_troughs, 
#                              coords={'occurrences': np.arange(1,idx_troughs.shape[0]+1),
#                                      'lat': latitude,
#                                      'lon': longitude}, 
#                              dims=['occurrences','lat','lon'])
                   
    
    
#     idx_peaks.to_netcdf(path='data/processed/range-peak-indices.nc')
#     idx_troughs.to_netcdf(path='data/processed/range-trough-indices.nc')
    
    
    
        
        
    indices = xr.Dataset({
        'peak': xr.DataArray(
            data=idx_peaks,
            coords={'p_occur': np.arange(1,idx_peaks.shape[0]+1),
                    'lat': latitude,
                    'lon': longitude},
            dims=['p_occur','lat','lon']),
        
        'trough': xr.DataArray(
            data=idx_troughs,
            coords={'t_occur': np.arange(1,idx_troughs.shape[0]+1),
                    'lat': latitude,
                    'lon': longitude},
            dims=['t_occur','lat','lon'])})
                                   

    indices.to_netcdf(path=f'data/processed/{filename}')
   

In [ ]:
start = time()
idx_peaks,idx_troughs = peak_trough_idx(range_combined.zos.values,range_combined.time.values,range_combined.lat.values,range_combined.lon.values)
write_peak_trough(idx_peaks,idx_troughs,range_combined.lat.values,range_combined.lon.values,'range-peak-trough-indices.nc')
end = time()
print(end-start)